In [1]:
import numpy as np
from functions import *
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from mlp import MLP

# Hyperparameters
batch_size = 250  # Batch size for mini-batch gradient descent
learning_rate = 0.1  # Learning rate for weight updates
epochs = 10  # Number of epochs to train
input_size = 784  # Input size (28x28 flattened images)
output_size = 10  # Number of output neurons (10 classes for MNIST)

# Load MNIST dataset and apply transformations (convert to tensor, normalize)
transform = transforms.Compose([
    transforms.ToTensor(),  # Convert PIL image to tensor
    transforms.Normalize((0.5,), (0.5,))  # Optional: normalize the data
])

train_dataset = datasets.MNIST(
    root='./data', 
    train=True, 
    transform=transform, 
    download=True
)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

# Initialize MLP model
mlp = MLP(learning_rate=learning_rate)
j=0
# Training loop
for epoch in range(epochs):
    total_loss = 0  # Variable to store total loss for the epoch
    
    for batch_idx, (batch_images, batch_labels) in enumerate(train_loader):
        batch_images = batch_images.view(batch_size, -1)  # Flatten images to 1D vector (28x28 = 784)
        batch_labels = np.eye(output_size)[batch_labels]  # One-hot encode the labels

        # Perform forward pass: Get model predictions
        predictions = mlp.forward(batch_images.numpy())

        # Compute loss (Mean Squared Error)
        loss = cross_entropy_loss(batch_labels, predictions)
        total_loss += loss
        
        # Perform backpropagation
        gradients_w, gradients_b = mlp.back_propagation(batch_images.numpy(), batch_labels)

        # Update weights and biases
        mlp.update_weights(gradients_w, gradients_b)

    # Print loss after every epoch
    print(f"Epoch {epoch + 1}/{epochs}, Loss: {total_loss / len(train_loader)}")

# After training is complete, you can save the model or evaluate it on the test set.

/opt/anaconda3/envs/computer_vision/lib/python3.8/site-packages/torchvision/io/image.py:14: UserWarning: Failed to load image Python extension: 'dlopen(/opt/anaconda3/envs/computer_vision/lib/python3.8/site-packages/torchvision/image.so, 0x0006): Library not loaded: @rpath/libjpeg.9.dylib
  Referenced from: <CAF361F5-1CAC-3EBE-9FC4-4B823D275CAA> /opt/anaconda3/envs/computer_vision/lib/python3.8/site-packages/torchvision/image.so
  Reason: tried: '/opt/anaconda3/envs/computer_vision/lib/python3.8/site-packages/torchvision/../../../libjpeg.9.dylib' (no such file), '/opt/anaconda3/envs/computer_vision/lib/python3.8/site-packages/torchvision/../../../libjpeg.9.dylib' (no such file), '/opt/anaconda3/envs/computer_vision/lib/python3.8/lib-dynload/../../libjpeg.9.dylib' (no such file), '/opt/anaconda3/envs/computer_vision/bin/../lib/libjpeg.9.dylib' (no such file)'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be somet

Epoch 1/10, Loss: 2.3013574998823483
Epoch 2/10, Loss: 2.0459628969232075
Epoch 3/10, Loss: 1.0098361211759272
Epoch 4/10, Loss: 0.4786184357504235
Epoch 5/10, Loss: 0.3674801770022605
Epoch 6/10, Loss: 0.31567782864757327
Epoch 7/10, Loss: 0.27142595363652916
Epoch 8/10, Loss: 0.2535417847731933
Epoch 9/10, Loss: 0.23150305866575643
Epoch 10/10, Loss: 0.22076060730309235
